<a href="https://colab.research.google.com/github/umerhasan17/NLPzoo/blob/master/text-summarisation/keyword_extraction_TextRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TextRank

TextRank is an algorithm based off of PageRank which is used to extract keywords from bodies of text.

Original source code from [Bramble Xu](https://gist.github.com/BrambleXu/3d47bbdbd1ee4e6fc695b0ddb88cbf99#file-textrank4keyword-py)



In [ ]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    # Used to add custom stopwords in addition to ones provided by spacy
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [ ]:
text = '''Christmas was coming. One morning in mid-December, Hogwarts woke to 
find itself covered in several feet of snow. The lake froze solid and the 
Weasley twins were punished for bewitching several snowballs so that they 
followed Quirrell around, bouncing off the back of his turban. The few owls 
that managed to battle their way through the stormy sky to deliver mail had 
to be nursed back to health by Hagrid before they could fly off again.\n\tNo 
one could wait for the holidays to start. While the Gryffindor common room 
and the Great Hall had roaring fires, the drafty corridors had become icy 
and a bitter wind rattled the windows in the classrooms. Worst of all were 
Professor Snape\'s classes down in the dungeons, where their breath rose in 
a mist before them and they kept as close as possible to their hot cauldrons.
\n\t\"I do feel so sorry,\" said Draco Malfoy, one Potions class, \"for all 
those people who have to stay at Hogwarts for Christmas because they\'re not 
wanted at home.\"\n\tHe was looking over at Harry as he spoke. Crabbe and Goyle 
chuckled. Harry, who was measuring out powdered spine of lionfish, ignored them. 
Malfoy had been even more unpleasant than usual since the Quidditch match. 
Disgusted that the Slytherins had lost, he had tried to get everyone laughing 
at how a wide-mouthed tree frog would be replacing Harry as Seeker next. 
Then he\'d realized that nobody found this funny, because they were all so 
impressed at the way Harry had managed to stay on his bucking broomstick. 
So Malfoy, jealous and angry, had gone back to taunting Harry about having 
no proper family.''' 
    

In [10]:
tr = TextRank4Keyword()
tr.analyze(text, candidate_pos=['NOUN', 'PROPN'], window_size=4, lower=False)
tr.get_keywords(10)

Harry - 2.437478174603174
Hogwarts - 1.8691013888888888
Malfoy - 1.7596680284992785
way - 1.5346539862914863
dungeons - 1.3187027777777778
December - 1.225273611111111
people - 1.2223866161616161
class - 1.2163657828282828
twins - 1.1900458333333335
snowballs - 1.1900458333333335
fires - 1.1774375
corridors - 1.1774375


In [ ]:
way - 1.6203819444444445
corridors - 1.3187027777777778
drafty - 1.1386916666666667
wind - 1.1386916666666667
dungeons - 1.0814583333333332
breath - 1.0814583333333332
mist - 1.0814583333333332
class - 1.0814583333333332
people - 1.0814583333333332
Hogwarts - 1.0814583333333332
sky - 1.065815972222222
mail - 1.065815972222222